# Librerias

In [31]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import plotly.graph_objs as go
from collections import defaultdict
import statistics
import json
import re
import matplotlib.pyplot as plt


# Codigo de main

In [32]:
#TAMBIEN DEVUELVE LOS INTENTOS INDIVIDUALES DE CADA JUGADOR PARA COMPLETAR CADA NIVEL
def getIntentosMedios_HastaCompletarNivel(intentosNecesarios, intentosOrdenados = False):
    intentosMedios = defaultdict(list)
    intentosCompletarNivelIndividual = defaultdict(defaultdict)
    
    for name in intentosNecesarios:
        for level in intentosNecesarios[name]:
            cont = 0
            for i in intentosNecesarios[name][level]:
                cont += i["intentos"]
                if i["success"] == True:
                    intentosMedios[level].append(cont)
                    intentosCompletarNivelIndividual[name][level] = cont
                    break
            
    for level in intentosMedios:
        intentosMedios[level] = round(statistics.mean(intentosMedios[level]), 2)
    
    if intentosOrdenados:
        intentosMedios = sorted(intentosMedios.items(), key=lambda x: x[1])
    else:
        intentosMedios = list(intentosMedios.items())
    return {"intentosMedios" : intentosMedios, "intentosIndividual" : intentosCompletarNivelIndividual}

def extraerTiemposPorNivelJugador(rawData):
    
    tiempos = defaultdict(defaultdict)
    intentosNecesarios = defaultdict(defaultdict)
    
    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
    
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    
    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
        objectId = evento["object"]["id"]
        
        if erLevel.search(obj): #Si el objeto de la acción es un nivel
            levelCode = erIdLevel.split(objectId)[-1]
            if levelCode != "editor_level":
                if erInitialized.search(verb): #Si la acción es inicio o reinicio
                    if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                        if levelCode in tiempos[name]:
                            intentosNecesarios[name][levelCode].append({"intentos" : 1, "success" : False})
                            tiempos[name][levelCode].append({"ini" : timestamp, "fin" : None, "stars" : ""})
                        else:
                            intentosNecesarios[name][levelCode] = [{"intentos" : 1, "success" : False}]
                            tiempos[name][levelCode] = [{"ini" : timestamp, "fin" : None, "stars" : ""}]
                    else:
                        intentosNecesarios[name][levelCode][-1]["intentos"] += 1
                elif erCompleted.search(verb):
                    if evento["result"]["score"]["raw"] > 0 :
                        if levelCode in tiempos[name]:
                            intentosNecesarios[name][levelCode][-1]["success"] = True
                            tiempos[name][levelCode][-1]["fin"] = timestamp
                            tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]

                    elif evento["result"]["score"]["raw"] == -1:
                        if levelCode in tiempos[name]:
                            tiempos[name][levelCode][-1]["fin"] = timestamp
                            tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
    
    return {"tiempos" : tiempos, "intentosNecesarios" : intentosNecesarios}

JSONFile = open('./data/trazasOrdenadas.json')
rawData = json.load(JSONFile)
JSONFile.close()

intentosOrdenados = False
resultados_Tiempos_Nivel_Jugador = extraerTiemposPorNivelJugador(rawData)
intentosMedios_Individual = getIntentosMedios_HastaCompletarNivel(resultados_Tiempos_Nivel_Jugador["intentosNecesarios"], intentosOrdenados)
#pd.DataFrame(intentosMedios_Individual["intentosMedios"])
#pd.DataFrame(intentosMedios_Individual["intentosIndividual"])

In [33]:
filename = 'results-survey976727_pre.csv'
data = pd.read_csv(filename, sep=';', header=0)

data.drop(['submitdate', 'G00Q01[SQ01]', 'G02Q01', 'G02Q02', 'G02Q03', 'G02Q04', 'G03Q02[other]', 'id'], axis = 'columns', inplace=True)

#data

In [34]:
# Crear un DataFrame con los datos de los intentos medios de cada jugador

intentosJugadores = pd.DataFrame(intentosMedios_Individual["intentosIndividual"])
#intentosJugadores

# Convertir columnas en filas
df_stacked = intentosJugadores.stack().reset_index()
df_stacked.columns = ['nivel', 'token', 'intentos']

# Calcular promedio de intentos por jugador
promedio_intentos = df_stacked.groupby('token')['intentos'].mean()

# Crear DataFrame final
df_final = pd.DataFrame({'token': promedio_intentos.index, 'intentos': promedio_intentos.values})

df_global = pd.merge(df_final, data, on='token')
#df_global


In [35]:
# Calcular los límites de los valores atípicos
# regla empírica, que establece que los valores atípicos se encuentran a más de 3 desviaciones estándar de la media.

Q1_edad = df_global['G03Q01'].quantile(0.25)
Q3_edad = df_global['G03Q01'].quantile(0.75)
IQR_edad = Q3_edad - Q1_edad
outlier_threshold_edad = Q3_edad + 1.5*IQR_edad

Q1_intentos = df_global['intentos'].quantile(0.25)
Q3_intentos = df_global['intentos'].quantile(0.75)
IQR_intentos = Q3_intentos - Q1_intentos
outlier_threshold_intentos = Q3_intentos + 1.5*IQR_intentos

df = df_global[(df_global['G03Q01'] <= outlier_threshold_edad) & (df_global['intentos'] <= outlier_threshold_intentos)]

df['genero_num'] = df['G03Q02'].map({'Masculino': 0, 'Femenino': 1, 'Otro': 2})


#df


C:\Users\david\AppData\Local\Temp\ipykernel_25380\1715381978.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [36]:
fig = go.Figure(data=[go.Scatter3d(
    x=df['G03Q01'],
    y=df['genero_num'],
    z=df['intentos'],
    mode='markers',
    marker=dict(
        size=5,
        color='blue',
        opacity=0.8
    )
)])

fig.update_layout(scene=dict(
    xaxis_title='Edad',
    yaxis_title='Genero',
    zaxis_title='Intentos'),
    width=700,
    margin=dict(r=20, b=10, l=10, t=10))

fig.show()




# con KMeans:

In [37]:
# from sklearn.cluster import KMeans

# Eliminamos la columna de 'genero' antes de crear la matriz de características
# intentos_mat = df.drop(['G03Q02', 'token'], axis=1).values

# Definimos o número de iteracións
# n_iter = 20

# Obtemos os mesmos centroides iniciais que antes
# SEED_VALUE = 190463  # NON CAMBIES ESTE SEED, así todos teremos os mesmos resultados
# np.random.seed(SEED_VALUE)
# centr_iniciais = intentos_mat[np.random.choice(intentos_mat.shape[0], size=5, replace=False)]

# Definimos o modelo
# model = KMeans(n_clusters=len(centr_iniciais), init=centr_iniciais, n_init=1,
               # max_iter=n_iter, algorithm='full', random_state=SEED_VALUE)

# Axustamos o modelo aos datos
# np.random.seed(SEED_VALUE)
# agrupamento = model.fit(intentos_mat)


# con DBSCAN:

In [38]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Eliminamos la columna de 'genero' antes de crear la matriz de características
#intentos_mat = df.drop(['G03Q02', 'token'], axis=1).values

# Definimos el modelo
#model = DBSCAN(eps=0.5, min_samples=5)

# Ajustamos el modelo a los datos
# agrupamento = model.fit(intentos_mat)


In [39]:
# Creamos la figura
#fig = go.Figure(data=[go.Scatter3d(x=df['G03Q01'], y=df['genero_num'], z=df['intentos'], mode='markers', 
                                   #marker=dict(size=3, color=agrupamento.labels_ + 1, opacity=0.8, 
                                               #colorscale='Viridis'))])

# Agregamos la barra de colores
#fig.update_layout(title='Clustering de intentos',
                  #scene=dict(xaxis_title='Edad', yaxis_title='Genero', zaxis_title='Intentos'),
                  #coloraxis=dict(colorscale='Viridis', colorbar=dict(title='Cluster')))

# Mostramos la figura
#fig.show()

# Con Hierarchical Agglomerative Clustering (HAC):

In [43]:
import numpy as np
import plotly.graph_objects as go
from scipy.cluster.hierarchy import linkage, fcluster

# Eliminamos la columna de 'genero' antes de crear la matriz de características
# intentos_mat = df.drop(['G03Q02', 'token'], axis=1).values

# Aplicamos HAC para obtener los clusters
# Z = linkage(intentos_mat, 'ward')
# agrupamiento = fcluster(Z, t=6, criterion='maxclust')

# Creamos un diccionario que asigne un color aleatorio a cada cluster
# colores = {}
#for cluster in set(agrupamiento):
    #colores[cluster] = f'rgb({np.random.randint(0, 256)}, {np.random.randint(0, 256)}, {np.random.randint(0, 256)})'

# Creamos la figura
#fig = go.Figure(data=[go.Scatter3d(x=df['G03Q01'], y=df['genero_num'], z=df['intentos'], mode='markers', 
                                   #marker=dict(size=3, color=[colores[cluster] for cluster in agrupamiento], 
                                               #opacity=0.8))])

# Agregamos la barra de colores
#fig.update_layout(title='Clustering de intentos',
                  #scene=dict(xaxis_title='Edad', yaxis_title='Genero', zaxis_title='Intentos'),
                  #coloraxis=dict(colorbar=dict(title='Cluster')))

# Mostramos la figura
#fig.show()


In [47]:
from sklearn.cluster import SpectralClustering

# Eliminamos la columna de 'genero' antes de crear la matriz de características
intentos_mat = df.drop(['G03Q02', 'token'], axis=1).values

# Definimos el modelo
model = SpectralClustering(n_components=None, random_state=SEED_VALUE)

# Axustamos el modelo aos datos
agrupamiento = model.fit_predict(intentos_mat)

# Creamos un diccionario que asigne un color aleatorio a cada cluster
colores = {}
for cluster in set(agrupamiento):
    colores[cluster] = f'rgb({np.random.randint(0, 256)}, {np.random.randint(0, 256)}, {np.random.randint(0, 256)})'

# Creamos la figura
fig = go.Figure(data=[go.Scatter3d(x=df['G03Q01'], y=df['genero_num'], z=df['intentos'], mode='markers', 
                                   marker=dict(size=3, color=[colores[cluster] for cluster in agrupamiento], 
                                               opacity=0.8))])

# Agregamos la barra de colores
fig.update_layout(title='Clustering de intentos',
                  scene=dict(xaxis_title='Edad', yaxis_title='Genero', zaxis_title='Intentos'),
                  coloraxis=dict(colorbar=dict(title='Cluster')))

# Mostramos la figura
fig.show()


In [48]:
df['cluster'] = agrupamento.labels_
df

C:\Users\david\AppData\Local\Temp\ipykernel_25380\2127888770.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,token,intentos,G03Q01,G03Q02,genero_num,cluster
0,accul,3.650000,14,Masculino,0,1
1,ahxym,2.875000,14,Otro,2,-1
2,aotlm,2.250000,15,Masculino,0,0
3,asopk,1.885714,14,Masculino,0,1
4,atlzs,2.423077,14,Femenino,1,2
...,...,...,...,...,...,...
139,zbgwk,4.933333,15,Otro,2,-1
140,zcibf,1.695652,13,Femenino,1,5
141,zdnso,1.925926,15,Femenino,1,6
143,zwzfv,3.043478,14,Masculino,0,1


# Anteriores

In [49]:


# Pivotear el DataFrame para tener un formato largo
intentos_df_long = pd.melt(intentosJugadores.reset_index(), id_vars=['index'], var_name='Jugador', value_name='Intentos')
intentos_df_long.columns = ['Nivel', 'Jugador', 'Intentos']

# Agrupar los intentos de cada jugador por nivel y tomar la media
intentos_jugador_nivel = intentos_df_long.groupby(['Jugador', 'Nivel'])['Intentos'].mean().unstack()

# Unir la tabla de intentos de cada jugador con los datos originales
df = data_filtrado.merge(intentos_jugador_nivel, left_on='token', right_index=True)

# Seleccionar las variables a utilizar para el clustering
X = df.iloc[:, 3:] # Seleccionar las columnas correspondientes a los niveles

# Imputar los valores faltantes
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Normalizar las variables utilizando StandardScaler
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_imputed)

# Definir el número de clusters y entrenar el modelo de K-means
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(X_norm)

# Asignar a cada jugador el cluster correspondiente
clusters = kmeans.labels_
df['Cluster'] = clusters

# Calcular la media de los números de intentos en cada cluster
means = df.groupby(['G03Q02', 'G03Q01', 'Cluster'])[X.columns].mean()

# Mostrar los resultados
print(means) #Dar a abrir en un editor de texto para ver mejor la tabla
#intentosJugadores


NameError: name 'data_filtrado' is not defined

In [ ]:
tiemposJugadores = pd.DataFrame(tiemposMedios["tiemposIndividuales"])

# Aplicar la función int() solamente a los valores que no son NaN en el DataFrame
tiemposJugadores = tiemposJugadores.applymap(lambda x: int(x) if x == x else x)

# Pivotear el DataFrame para tener un formato largo
tiempos_df_long = pd.melt(tiemposJugadores.reset_index(), id_vars=['index'], var_name='Jugador', value_name='Intentos')
tiempos_df_long.columns = ['Nivel', 'Jugador', 'Intentos']

# Agrupar los intentos de cada jugador por nivel y tomar la media
tiempos_jugador_nivel = tiempos_df_long.groupby(['Jugador', 'Nivel'])['Intentos'].mean().unstack()

# Unir la tabla de intentos de cada jugador con los datos originales
df = data_filtrado.merge(tiempos_jugador_nivel, left_on='token', right_index=True)

# Seleccionar las variables a utilizar para el clustering
X = df.iloc[:, 3:] # Seleccionar las columnas correspondientes a los niveles

# Imputar los valores faltantes
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Normalizar las variables utilizando StandardScaler
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_imputed)

# Definir el número de clusters y entrenar el modelo de K-means
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(X_norm)

# Asignar a cada jugador el cluster correspondiente
clusters = kmeans.labels_
df['Cluster'] = clusters

# Calcular la media de los números de intentos en cada cluster
means = df.groupby(['G03Q02', 'G03Q01', 'Cluster'])[X.columns].mean()

# Mostrar los resultados
print(means)
# tiemposJugadores


In [ ]:
media_intentos = df.groupby(['G03Q02', 'G03Q01', 'Cluster'])[X.columns].mean().reset_index()
media_intentos = pd.melt(media_intentos, id_vars=['G03Q02', 'G03Q01', 'Cluster'], var_name='Nivel', value_name='Intentos')

import seaborn as sns
import matplotlib.pyplot as plt

# Crear una figura para cada nivel
for nivel in intentos_df_long['Nivel'].unique():
    # Seleccionar los datos del nivel y generar una tabla pivote con los valores medios de intentos por género
    df_nivel = media_intentos[media_intentos['Nivel'] == nivel]
    pivot_table = pd.pivot_table(df_nivel, values='Intentos', index=['G03Q01'], columns=['Cluster'], aggfunc=np.mean)
    
    # Crear un gráfico de barras apiladas para comparar los valores medios de intentos por género en cada cluster
    sns.catplot(x='G03Q01', y='Intentos', hue='Cluster', data=df_nivel, kind='bar', ci=None, legend_out=False)
    plt.title(f'Nivel {nivel}')
    plt.show()
